In [ ]:
import duckdb
import  pandas as pd


import os
import sys
from typing_extensions import TypedDict, Annotated

sys.path.extend([os.getcwd(),'..'])

from  config import  CONFIG

# import nltk
# from nltk.tokenize import word_tokenize, sent_tokenize
# from nltk.metrics.distance  import edit_distance
# nltk.download('punkt_tab')
# nltk.download('words')
# from nltk.corpus import words
# correct_words = words.words()

import os
from langchain.chat_models import init_chat_model

with open("../openrouter_key.txt","r") as f:
    key=f.read()

llm = init_chat_model(model_provider=CONFIG.LLM_MODEL_PROVIDER,
                      model=CONFIG.LLM_MODEL,
                      base_url=CONFIG.LLM_BASE_URL,
                      api_key=key)

sep="\n#@#@#@#@#@#@#@#@#@##@#@#@#@#@#@#@#@#@##@#@#@#@#@#@#@#@#@#\n"

In [ ]:
llm.invoke("hello").content

In [ ]:
with duckdb.connect('../data.duckdb') as con:
    data=con.execute("SELECT * FROM papers_raw;").df()

In [ ]:
# [print(sep,t[:150]) for t in data.raw_text];


In [ ]:
text=data.raw_text[2]

In [ ]:
print(text[:1500])

In [ ]:
prompt=f"""
The text enclosed in triple backticks is  the header of a research paper.
Extract the title from given text.
Return only the title.

```{text[:200]}```
"""
print(llm.invoke(prompt).content.strip())

In [ ]:
class ResearchPaper(TypedDict):
    """A research  paper  with details."""
    title: Annotated[str, ..., "The title of the paper"]
    #abstract_summary: Annotated[str, ..., "Summary of the abstract of the paper"]
    abstract: Annotated[str, ..., "The abstract of the paper"]
    
text=data.raw_text[6]

prompt=f"""
The text enclosed in triple backticks is part of the first page of a research paper.
Extract the paper title and abstract from the given text.
Fix any grammar or syntax errors.

```{text[:1500]}```
"""
llm_with_structure = llm.with_structured_output(ResearchPaper)

print(llm_with_structure.invoke(prompt))